In [75]:
import numpy as np
import pandas as pd

In [76]:
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import OrdinalEncoder

In [77]:
df=pd.read_csv('covid_toy.csv')

In [78]:
df.head()

,age,gender,fever,cough,city,has_covid
0,60,Male,103.0,Mild,Kolkata,No
1,27,Male,100.0,Mild,Delhi,Yes
2,42,Male,101.0,Mild,Delhi,No
3,31,Female,98.0,Mild,Kolkata,No
4,65,Female,101.0,Mild,Mumbai,No


In [79]:
df['cough'].value_counts()

cough
Mild      62
Strong    38
Name: count, dtype: int64

In [80]:
print(df['city'].value_counts())

city
Kolkata      32
Bangalore    30
Delhi        22
Mumbai       16
Name: count, dtype: int64


In [81]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test = train_test_split(df.drop(columns=['has_covid']),df['has_covid'],
                                                test_size=0.2)

In [82]:
X_train

,age,gender,fever,cough,city
65,69,Female,102.0,Mild,Bangalore
88,5,Female,100.0,Mild,Kolkata
32,34,Female,101.0,Strong,Delhi
70,68,Female,101.0,Strong,Delhi
41,82,Male,NaN,Mild,Kolkata
...,...,...,...,...,...
56,71,Male,NaN,Strong,Kolkata
91,38,Male,NaN,Mild,Delhi
74,34,Female,104.0,Strong,Delhi
21,73,Male,98.0,Mild,Bangalore


1. Without Column Transformer

In [83]:
# adding simple imputer to fever col
si=SimpleImputer()
X_train_fever=si.fit_transform(X_train[['fever']])
X_test_fever = si.fit_transform(X_test[['fever']])
X_test_fever.shape

(20, 1)

In [84]:
# Ordinalencoding -> cough
oe=OrdinalEncoder(categories=[['Mild','Strong']])
X_train_cough=oe.fit_transform(X_train[['cough']])
X_test_cought=oe.fit_transform(X_test[['cough']])
print(X_train_cough)


[[0.]
 [0.]
 [1.]
 [1.]
 [0.]
 [1.]
 [0.]
 [0.]
 [1.]
 [0.]
 [0.]
 [1.]
 [0.]
 [0.]
 [1.]
 [0.]
 [1.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [1.]
 [0.]
 [0.]
 [1.]
 [1.]
 [0.]
 [0.]
 [0.]
 [0.]
 [1.]
 [0.]
 [0.]
 [0.]
 [1.]
 [0.]
 [0.]
 [0.]
 [1.]
 [0.]
 [1.]
 [1.]
 [0.]
 [1.]
 [0.]
 [1.]
 [0.]
 [1.]
 [1.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [1.]
 [0.]
 [1.]
 [0.]
 [0.]
 [1.]
 [1.]
 [1.]
 [0.]
 [1.]
 [1.]
 [0.]
 [1.]
 [0.]
 [1.]
 [1.]
 [0.]
 [1.]
 [0.]
 [0.]]


In [85]:
print("X_test_cought\n",X_test_cought)

X_test_cought
 [[1.]
 [0.]
 [0.]
 [0.]
 [1.]
 [1.]
 [0.]
 [0.]
 [1.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [1.]
 [1.]
 [0.]
 [1.]
 [1.]]


In [86]:
# OneHotEncoding -> gender,city
ohe = OneHotEncoder(drop='first',sparse_output=False)
X_train_gender_city = ohe.fit_transform(X_train[['gender','city']])

# also the test data
X_test_gender_city = ohe.fit_transform(X_test[['gender','city']])

X_train_gender_city.shape

(80, 4)

In [87]:
#Extracting Age
X_train_age=X_train['age'].values

X_test_age=X_test['age'].values

In [88]:
X_train_age.reshape(80,1)

array([[69],
       [ 5],
       [34],
       [68],
       [82],
       [46],
       [83],
       [19],
       [40],
       [65],
       [19],
       [34],
       [16],
       [27],
       [82],
       [14],
       [13],
       [48],
       [42],
       [69],
       [80],
       [65],
       [72],
       [15],
       [49],
       [22],
       [60],
       [14],
       [70],
       [ 8],
       [25],
       [10],
       [44],
       [75],
       [84],
       [49],
       [19],
       [24],
       [38],
       [11],
       [74],
       [34],
       [ 5],
       [ 5],
       [47],
       [83],
       [51],
       [55],
       [33],
       [60],
       [10],
       [54],
       [12],
       [42],
       [26],
       [17],
       [51],
       [65],
       [27],
       [18],
       [20],
       [75],
       [79],
       [ 6],
       [65],
       [42],
       [19],
       [69],
       [80],
       [71],
       [59],
       [31],
       [66],
       [81],
       [12],
       [71],
       [38],

In [89]:
# # Extracting Age
# X_train_age = X_train.drop(columns=['gender','fever','cough','city']).values

# # also the test data
# X_test_age = X_test.drop(columns=['gender','fever','cough','city']).values

# X_train_age

2. with Column Transformer

In [90]:
from sklearn.compose import ColumnTransformer
transformer = ColumnTransformer(transformers=[
    ('tnf1',SimpleImputer(),['fever']),
    ('tnf2',OrdinalEncoder(categories=[['Mild','Strong']]),['cough']),
    ('tnf3',OneHotEncoder(sparse_output=False,drop='first'),['gender','city'])
    ],remainder='passthrough')

In [91]:
X_train_trans=transformer.fit_transform(X_train)
X_train=pd.DataFrame(X_train_trans)
X_train.columns=['Fever','gender','cough','city_delhi','city_kolkata','city_banglore','age']

In [92]:
X_train

,Fever,gender,cough,city_delhi,city_kolkata,city_banglore,age
0,102.000000,0.0,0.0,0.0,0.0,0.0,69.0
1,100.000000,0.0,0.0,0.0,1.0,0.0,5.0
2,101.000000,1.0,0.0,1.0,0.0,0.0,34.0
3,101.000000,1.0,0.0,1.0,0.0,0.0,68.0
4,100.833333,0.0,1.0,0.0,1.0,0.0,82.0
...,...,...,...,...,...,...,...
75,100.833333,1.0,1.0,0.0,1.0,0.0,71.0
76,100.833333,0.0,1.0,1.0,0.0,0.0,38.0
77,104.000000,1.0,0.0,1.0,0.0,0.0,34.0
78,98.000000,0.0,1.0,0.0,0.0,0.0,73.0
